<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<b>DiaObject Sample Identification</b> <br>
Contact author: Melissa Graham <br>
Last verified to run: <i>yyyy-mm-dd</i> <br>
LSST Science Piplines version: Weekly <i>yyyy_xx</i> <br>
Container Size: medium <br>
Targeted learning level: intermediate <br>

In [ ]:
%load_ext pycodestyle_magic
%flake8_on
import logging
logging.getLogger("flake8").setLevel(logging.FATAL)

**Description:** To use the `DiaObject` catalog parameters to identify samples of time-variable objects of interest.

**Skills:** Use the TAP service. Use the DP0.2 time-domain catalogs.

**LSST Data Products:** DP0.2 `DiaObject` and `DiaSource` table data.

**Packages:** lsst.rsp, astropy.cosmology, numpy, matplotlib

**Credit:** Originally developed by Melissa Graham and the Rubin Community Engagement Team for Data Preview 0. Please consider acknowledging them if this notebook is used for the preparation of journal articles, software releases, or other notebooks.

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0-1.lsst.io">dp0-1.lsst.io</a>. Questions are welcome as new topics in the <a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> of the Rubin Community Forum. Rubin staff will respond to all questions posted there.

## 1. Introduction

This notebook guides the use of DiaObject summary parameters to identify a sample of objects of interest for further study.

As an example, a sample of potential low-redshift, well-sampled Type Ia supernovae are identified. 

This notebook uses the DiaObject lightcurve summary parameters that were introduced in <b>NB REF HERE</b>.

Learn more about the contents of the DiaObject and DiaSource tables in the **(Placeholder <a href="https://dp0-1.lsst.io/data-products-dp0-1/index.html#dp0-1-data-products-definition-document-dpdd">DP0.2 Data Products Definitions Documentation</a>)**.

**Caveats:** The DP0.2 `DiaObject` table is missing some variability characterization parameters (<a href="https://dmtn-118.lsst.io">DMTN-118</a>) and host association parameters (<a href="https://dmtn-151.lsst.io">DMTN-151</a>) which will be added in the future.

### 1.1 Package Imports

 * lsst.rsp -- The LSST Science Pipelines package for RSP functionality such as the TAP service (<a href="http://pipelines.lsst.io">pipelines.lsst.io</a>).
 * astropy.cosmology -- An open-source package of cosmology tools (<a href="https://docs.astropy.org/en/stable/cosmology/index.html"> the astropy cosmology documentation</a>).
 * numpy -- A package for scientific computing with arrays in Python (<a href="https://numpy.org">numpy.org</a>).
 * matplotlib -- A package for creating plots (<a href="http://matplotlib.org">matplotlib.org</a>).


In [ ]:
import numpy
import matplotlib.pyplot as plt
# plt.style.use('tableau-colorblind10')

import time

from lsst.rsp import get_tap_service

from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

### 1.2 Define Functions and Parameters

Start the TAP service.

In [ ]:
service = get_tap_service()

## 2. Establish Query Parameters

### 2.1. apparent r-band magnitudes

Define the desired redshift boundaries.

Use the astropy.cosmology package to convert redshift to distance modulus.

Define the range of peak apparent r-band magnitudes, assuming that Type Ia supernovae have an intrinsic brightness of about -19 magnitudes.

In [ ]:
redshift_min = 0.1
redshift_max = 0.3

snia_peak_mag = -19.0
snia_peak_mag_range = 0.5

snia_peak_mr_min = cosmo.distmod(redshift_min).value + snia_peak_mag - snia_peak_mag_range
snia_peak_mr_max = cosmo.distmod(redshift_max).value + snia_peak_mag + snia_peak_mag_range

print('Min and max apparent r-band magnitudes are %5.2f and %5.2f mag.' %
      (snia_peak_mr_min, snia_peak_mr_max))

### 2.2. apparent g- and i-band magnitudes

Define maximum magnitudes in the g- and i-bands to enforce detection in at least the three filters g, r, and i.

In [ ]:
snia_peak_mg_max = 24.0
snia_peak_mi_max = 24.0

### 2.3. number of DiaSources

The goal was to identify potential _well-sampled_ Type Ia supernovae, so define the minimum number of lightcurve points (number of DiaSources).

In [ ]:
nDiaSources_min = 15

## 3. Query the DiaObjects Table

The query may take a few tens of seconds.

When query completes, transfer the results to an astropy table.

In [ ]:
results = service.search("SELECT ra, decl, diaObjectId, nDiaSources, "
                         "scisql_fluxToAbMag(gPSFluxMax/1e32) AS gMagMin, "
                         "scisql_fluxToAbMag(rPSFluxMax/1e32) AS rMagMin, "
                         "scisql_fluxToAbMag(iPSFluxMax/1e32) AS iMagMin, "
                         "scisql_fluxToAbMag(gPSFluxMin/1e32) AS gMagMax, "
                         "scisql_fluxToAbMag(rPSFluxMin/1e32) AS rMagMax, "
                         "scisql_fluxToAbMag(iPSFluxMin/1e32) AS iMagMax "
                         "FROM dp02_test_PREOPS863_00.DiaObject "
                         "WHERE scisql_fluxToAbMag(rPSFluxMax/1e32) > "+str(snia_peak_mr_min)+" "
                         "AND scisql_fluxToAbMag(rPSFluxMax/1e32) < "+str(snia_peak_mr_max)+" "
                         "AND scisql_fluxToAbMag(gPSFluxMax/1e32) < "+str(snia_peak_mg_max)+" "
                         "AND scisql_fluxToAbMag(iPSFluxMax/1e32) < "+str(snia_peak_mi_max)+" "
                         "AND nDiaSources > "+str(nDiaSources_min)+" ",
                         maxrec=10000)
DiaObjs = results.to_table()
del results

Calculate the lightcurve amplitude in the r-filter.

In [ ]:
DiaObjs['rMagAmp'] = DiaObjs['rMagMax'] - DiaObjs['rMagMin']

Print the number of DiaObjects that were returned.

In [ ]:
print('The query returned ', len(DiaObjs), ' DiaObjects.')

Option to display the table

In [ ]:
# DiaObjs

Option to display histograms of the DiaObject parameters.

In [ ]:
# plt.hist(DiaObjs['rMagMin'], bins=20)
# plt.axvline(snia_peak_mr_min, color='magenta')
# plt.axvline(snia_peak_mr_max, color='magenta')
# plt.xlabel('Brightest Detected r-band Magnitude')
# plt.show()

In [ ]:
# plt.hist(DiaObjs['rMagAmp'], bins=20)
# plt.xlabel('Amplitude in r-band Magnitude')
# plt.show()

In [ ]:
# plt.hist(DiaObjs['nDiaSources'], bins=20)
# plt.xlabel('Number of Difference-Image Detections')
# plt.show()

## 4. Retrieve DiaSources

Get the difference-image photometry (lightcurves) from the DiaSource table.

### 4.1. caculate lightcurve duration

The sample of interest used in this notebook, low-redshift Type Ia supernovae, would be bright enough for detection (<24.5 mag) for up to one year.

Calculate the lightcurve duration -- the difference between the dates of last and first DiaSource -- for all of the DiaObjects returned.

In [ ]:
DiaObjs['duration'] = numpy.zeros(len(DiaObjs), dtype='float')

for j,DiaObjId in enumerate(DiaObjs['diaObjectId']):
    results = service.search("SELECT ra, decl, diaObjectId, diaSourceId, "
                             "filterName, midPointTai, psFlux, psFluxErr "
                             "FROM dp02_test_PREOPS863_00.DiaSource "
                             "WHERE diaObjectId = "+str(DiaObjId))
    results = results.to_table()
    DiaObjs['duration'][j] = numpy.max(results['midPointTai']) - numpy.min(results['midPointTai'])
    del results

Option to display a histogram of DiaObject durations.

In [ ]:
# plt.hist(DiaObjs['duration'], bins=20)
# plt.xlabel('Lightcurve Duration (Any Filter)')
# plt.show()

Option to plot duration versus r-band magnitude amplitude.

In [ ]:
# plt.plot(DiaObjs['duration'], DiaObjs['rMagAmp'], 'o')
# plt.xlabel('Lightcurve Duration (Any Filter)')
# plt.ylabel('Amplitude in r-band Magnitude')
# plt.show()

### 4.2. plot lightcurves of potential SNIa

Consider "potential SNIa" as the DiaObjects with durations < 1 year and with a magnitude amplitude >0.5 mag in the r-filter.

For plotting, define the filter names, colors, and symbols to be used.

In [ ]:
filter_names = ['u', 'g', 'r', 'i', 'z', 'y']
filter_color = ['darkviolet', 'darkgreen', 'red', 'darkorange', 'brown', 'black']
filter_symbol = ['o', '^', 'v', 's', '*', 'p']

The number of DiaObjects that have a duration of up to one year.

In [ ]:
tx = numpy.where((DiaObjs['duration'] < 365.) & (DiaObjs['rMagAmp'] > 0.5))[0]
print(len(tx))

Make the plot.

<br>
<br>

**WARNING**

**NONE OF THESE APPEAR TO BE TYPE Ia SUPERNOVAE???**

<br>
<br>


In [ ]:
fig, ax = plt.subplots(len(DiaObjs[tx]), figsize=(14,20), sharey=False, sharex=False)

for i, j in enumerate(tx):
    results = service.search("SELECT ra, decl, diaObjectId, diaSourceId, "
                             "filterName, midPointTai, "
                             "scisql_fluxToAbMag(psFlux/1e32) AS psAbMag "
                             "FROM dp02_test_PREOPS863_00.DiaSource "
                             "WHERE diaObjectId = "+str(DiaObjs['diaObjectId'][j]))
    results = results.to_table()

    for f, filt in enumerate(filter_names):
        fx = numpy.where(results['filterName'] == filt)[0]
        ax[i].plot(results['midPointTai'][fx], results['psAbMag'][fx], 
                   filter_symbol[f], ms=15, mew=0, alpha=0.5, color=filter_color[f])
        del fx
    
    ax[i].set_ylim([numpy.max(results['psAbMag'])+0.3, numpy.min(results['psAbMag'])-0.3])
    
    del results

## 5. Further Work

The next steps towards science with the sample of interest might include applying a lightcurve template fitter, or photometric classification codes, to the sample.

Another analysis option would be to use the DiaForcedSource table for the lightcurves, in order to include photometry measured below the detection limit.